In [ ]:
#Install the necessary packages

!pip install yfinance
!pip install matplotlib==3.5.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#Standard packages
import numpy as np
import pandas as pd

#Networks
import networkx as nx

#Dates
from datetime import datetime, timedelta
import time

#Finance packages
import yfinance as yf

#Statistics
from scipy.stats import t
from scipy.stats import skew, kurtosis
import random

#Plotting packages
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

from matplotlib.animation import FuncAnimation

from matplotlib import rcParams

rcParams["font.size"] = 20
rcParams["axes.labelsize"] = 30

rcParams["xtick.labelsize"] = 16
rcParams["ytick.labelsize"] = 16

rcParams["figure.figsize"] = (8,6)

# The Voter Model

The voter model is a mathematical model used to describe the dynamics of opinions or preferences in a population. It is typically used to model the spread of opinions or behaviors in a social network.

The model starts with a randomly assigned set of opinions or behaviors for each individual. At each time step, each individual independently adopts the opinion or behavior of one of its neighbors. The process continues until a consensus is reached, where all individuals in the population have the same opinion or behavior.

In this way, the voter model is a simple representation of how individuals can be influenced by the opinions or behaviors of their peers, leading to the emergence of a consensus.

Different types of networks can be considered for the underlying contact structure of the individuals, which controls who is the neighbor of who.

## Complete graph (fully connected network)

A complete graph, also known as a full graph or a universal graph, is a simple graph in which every pair of distinct vertices is connected by an edge. The degree distribution, $P(k)$ of a complete graph with $N$ nodes is given by

$$P(k)=\delta(k-(N-1))$$

this is, all nodes have degree $N-1$ since each node is connected to all others except themselves.

## Barabasi-Albert

The Barabasi-Albert (BA) model is a generative model for creating scale-free networks, which have a power-law degree distribution. The model begins with a small number of nodes, and at each time step, a new node is added to the network with a probability proportional to the degree (number of edges) of the existing nodes. The new node is then connected to $m$ existing nodes, where $m$ is a fixed parameter.

The probability that a node with degree k is chosen to be connected to the new node is given by the following probability mass function:

$$P_i = \frac{k_i}{\sum_{j=1}k_j}$$

The degree distribution of a network generated by the BA model follows a power-law distribution:

$$P(k) \propto k^{-\gamma}$$

with $\gamma > 0$.


## Watts-Strogatz

The Watts-Strogatz (WS) model is a generative model for creating small-world networks, which have a high degree of clustering and a small average shortest path length. The model begins with a regular lattice, where each node is connected to its k nearest neighbors. Then, each edge is rewired with probability $p$, meaning that it is disconnected from its original endpoint and reconnected to a randomly chosen node. For small values of p, the WS model creates networks that are highly clustered and have long average shortest path length. As p increases, the clustering decreases, and the average shortest path length decreases.

The degree distribution, $P(k)$ of a network generated by the WS model depends on the value of $p$, being a dirac Delta when $p=0$ and a Poisson distribution for $p=1$.


## Erdös–Rényi

The Erdös–Rényi model, also known as the binomial graph model, is a random graph model used in network theory. It is defined by the following parameters:

* $N$: Number of nodes in the graph.
* $p$: Probability that any two nodes are connected by an edge.

The model generates a graph $G(N,p)$ by randomly connecting each pair of nodes with probability $p$. The degree distribution, $P(k)$ , is given by

$$P(k) = \binom{N-1}{k}p_c^{k}(1-p_c)^{N-1-k}$$

Where $p_c=\frac{2M}{N(N-1)}$ and $M$ is the number of edges created in the graph.

# Animated example of the Voter Model

Run the following cell to compute an animation of the voter model in any of the previously mentioned networks. Feel free to change parameters and explore the results!

Note that each generative network model depends on different parameters (see the `init_model` function above). Thus, we must specify them in our `Voter_model_anim` function:

* Barabasi-Albert [(Networkx Documentation)](https://networkx.org/documentation/stable/reference/generated/networkx.generators.random_graphs.barabasi_albert_graph.html)

  * `m` - Number of edges to attach from a new node to existing nodes

* Watts-Strogatz [(Networkx Documentation)](https://networkx.org/documentation/stable/reference/generated/networkx.generators.random_graphs.watts_strogatz_graph.html)

  * `k` - Each node is joined with its k nearest neighbors in a ring topology.

  * `p`- The probability of rewiring each edge

* Erdös-Rényi [(Networkx Documentation)](https://networkx.org/documentation/stable/reference/generated/networkx.generators.random_graphs.erdos_renyi_graph.html)

  * `p` - Probability for edge creation

# Exercise 1. The average interface density of the Voter Model

* Compute the average interface density of the voter model for the 4 networks mentioned above.

* Comment the results that you obtain

**Note:** Use $N=200$ nodes, $T=100$ time steps and $M=100$ number of realisations. It will take a while, but should be fine.

**Clue:** *You can iterate through the edges of a network defined in Networkx using the `G.edges()` method, which returnes an EdgeView of the Graph, baiscally containing the nodes each edge connects*

**Clue:** *You can access the state of a node by using `G.nodes[idx]['state']*

**Clue:** *Compute the average interface density for each type of network at each iteration inside the loop of the $M$ realisations. In other words, **DO NOT** perform one loop for each type of network, it is inefficient.*

# Exercise 2: Implement the Imitation and Contrarian Behavior model

The Imitation and Contrarian Behavior model is an economic model that tries to explain how individuals make financial decisions based on the behavior of others and their own inclination to deviate from the crowd.

The model assumes that individuals observe the decisions made by others and tend to imitate the most successful ones, leading to herding behavior. However, some individuals, referred to as contrarians, choose to deviate from the crowd and make decisions that are contrary to the majority.

The model predicts that in the short-term, imitative behavior will dominate and drive prices away from their fundamental values, while contrarian behavior will eventually restore market efficiency and push prices back towards their fundamentals.

In mathematical terms, let's consider a market with $N$ agents and a stock price $P$. The agents can be in two states, $s=-1$ and $s=1$, corresponding to bullish (or optimistic) and bearish (or pessimistic), respectively. The model assumes that each agent $i$ is influenced by $m$ *other* agents and can either follow the majority decision of their neighbours of buying or selling the stock (imitation behavior), or take the opposite action (contrarian behavior). The decision of each agent is influenced by threshold parameters $\rho_{op}$, $\rho_{oo}$, $\rho_{po}$ and $\rho_{pp}$ that determine the extent to which each agent follows the majority or deviates from it.

The decision of an agent $i$ is given by the following rules:

* If the agent is bullish (optimistic, $s_i=1$)

  * If $\sum_{j\in m}s_i>m*\rho_{op}$

  * If

* If the agent is bearish (pessimistic, $S_i=-1$)

  * If

  * If